In [2]:
!pip install comet_ml

In [3]:
# import comet_ml at the top of your file
from comet_ml import Experiment

In [ ]:
# Create an experiment with your api key
experiment = Experiment(
    api_key="qPSB4ail2BLvJOkpLaqwE6FSe",
    project_name="unsupervised-learning",
    workspace="cebisiles",
)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
def load(s):
    l=[]
    for i in s:
        l+=[pd.read_csv("/kaggle/input/edsa-movie-recommendation-wilderness/"+i+".csv")]
    return l

In [4]:
def merge_movie_id(s):
    p=s[0]
    d=s[1:]
    for i in d:
        p=p.merge(i,how='left',on='movieId')
    return p

In [5]:
train_df=merge_movie_id(load(['train','movies','links','imdb_data','genome_tags','tags','genome_scores'])[:4])
test_df=merge_movie_id(load(['test','movies','links','imdb_data','genome_tags','tags','genome_scores'])[:4])

In [6]:
df=train_df.sample(100000)

In [7]:
from surprise import Reader, Dataset, SVD, NMF, SVDpp
from surprise.model_selection.validation import cross_validate
reader = Reader()
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)
svd = SVD()
svdpp = SVDpp()
nmf = NMF()

In [8]:
trainset = data.build_full_trainset()
svd.fit(trainset)
svdpp.fit(trainset)
nmf.fit(trainset)

In [9]:
l=[]
for index, row in test_df.iterrows():
    l+=[svdpp.predict(row['userId'], row['movieId']).est]

In [10]:
k=[]
for index, row in test_df.iterrows():
    k+=[svdpp.predict(row['userId'], row['movieId']).est]

In [11]:
m=[]
for index, row in test_df.iterrows():
    m+=[nmf.predict(row['userId'], row['movieId']).est]

In [12]:
sub=pd.read_csv("/kaggle/input/edsa-movie-recommendation-wilderness/sample_submission.csv")
sub2=pd.read_csv("/kaggle/input/edsa-movie-recommendation-wilderness/sample_submission.csv")
sub3=pd.read_csv("/kaggle/input/edsa-movie-recommendation-wilderness/sample_submission.csv")

In [13]:
sub['rating']=l

In [14]:
sub2['rating']=k

In [15]:
sub3['rating']=m

In [16]:
sub.to_csv("submission.csv",index=False)

In [17]:
sub2.to_csv("submission2.csv",index=False)

In [18]:
sub3.to_csv("submission3.csv",index=False)